**Web crawling using Beautiful soup ,sentiment analysis and readability analysis.**

Mounting the google drive.

In [ ]:
from google.colab import drive
drive.flush_and_unmount()


Drive not mounted, so nothing to flush and unmount.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive2




---


**Web crawling using Beautifulsoup on Input.xlsx url-ids.**

In [ ]:
# Import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import drive
import os

# Mount the Google Drive to save the output files
#drive.mount('/content/drive')

# Read the input Excel file
input_file = pd.read_excel('/content/drive/MyDrive/Input.xlsx')

# Define the output directory
output_dir = '/content/drive/MyDrive/URL_ID'

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through each URL and extract the article text
for index, row in input_file.iterrows():
    URL_ID = row['URL_ID']
    URL = row['URL']
    
    # Make a request to get the HTML content of the webpage
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract the article title and text from the HTML content
    title_tag = soup.find('title')  # find the title tag
    article_title = title_tag.string if title_tag else ''  # get the text inside the title tag, or set to empty string if title tag not found

    article_text = ''
    for paragraph in soup.find_all('p'):
        article_text += paragraph.get_text() + '\n'
    
    # Save the extracted article text to a text file
    output_file_path = os.path.join(output_dir, '{}.txt'.format(URL_ID))
    output_file = open(output_file_path, 'w')
    output_file.write(article_title + '\n' + article_text)
    output_file.close()




---


Stop words removal from the txt files and storing them inside google drive  in URL_ID2 folder.







In [ ]:
import os

# Define paths to input and output folders
URL_ID_path = '/content/drive/MyDrive/URL_ID'
stopwords_path = '/content/drive/MyDrive/BlackCOffer/StopWords'
filtered_path = '/content/drive/MyDrive/URL_ID2'

# Create the Filtered folder if it does not exist
if not os.path.exists(filtered_path):
    os.makedirs(filtered_path)

# Loop through each file in the URL_ID folder
for filename in os.listdir(URL_ID_path):
    if filename.endswith('.txt'):
        # Read the content of the file
        with open(os.path.join(URL_ID_path, filename), 'r') as file:
            text = file.read()
        
        # Remove stop words from the text
        stop_words = set()
        for filename in os.listdir(stopwords_path):
            if filename.endswith('.txt'):
                with open(os.path.join(stopwords_path, filename), encoding='utf-8') as file:
                    stop_words.update(file.read().splitlines())
        filtered_text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
        
        # Save the filtered text to a new file in the Filtered folder
        with open(os.path.join(filtered_path, filename), 'w') as file:
            file.write(filtered_text)




---

**Performing sentiment analysis on it.**

In [ ]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize

# Define paths for files and folders
stopwords_path = '/content/drive/MyDrive/BlackCOffer/StopWords'
positive_words_path = '/content/drive/MyDrive/BlackCOffer/MasterDictionary/positive-words.txt'
negative_words_path = '/content/drive/MyDrive/BlackCOffer/MasterDictionary/negative-words.txt'
input_folder_path = '/content/drive/MyDrive/URL_ID2'
output_folder_path = '/content/drive/MyDrive/Output'

# Create a set of stop words
stop_words = set()
for filename in os.listdir(stopwords_path):
    if filename.endswith('.txt'):
        with open(os.path.join(stopwords_path, filename), encoding='utf-8') as file:
            stop_words.update(file.read().splitlines())

# Create a dictionary of positive and negative words
positive_words = set()
with open(positive_words_path, encoding='utf-8') as file:
    for word in file.read().splitlines():
        if word.lower() not in stop_words:
            positive_words.add(word.lower())

negative_words = set()
with open(negative_words_path, encoding='utf-8') as file:
    for word in file.read().splitlines():
        if word.lower() not in stop_words:
            negative_words.add(word.lower())

# Create a DataFrame to store the results
columns = ['Filename', 'Positive Score', 'Negative Score', 'Polarity Score', 'Subjectivity Score']
results = pd.DataFrame(columns=columns)

# Loop through each file in the input folder
for filename in os.listdir(input_folder_path):
    if filename.endswith('.txt'):
        # Read the text from the file
        with open(os.path.join(input_folder_path, filename), encoding='utf-8') as file:
            text = file.read()
        
        # Tokenize the text and remove stop words
        tokens = [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words]
        
        # Calculate the positive and negative scores
        positive_score = sum([1 for token in tokens if token in positive_words])
        negative_score = -1 * sum([-1 for token in tokens if token in negative_words])
        
        # Calculate the polarity and subjectivity scores
        total_words = len(tokens)
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
        subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
        
        # Add the results to the DataFrame
        results = results.append({
            'Filename': filename,
            'Positive Score': positive_score,
            'Negative Score': negative_score,
            'Polarity Score': polarity_score,
            'Subjectivity Score': subjectivity_score
        }, ignore_index=True)

        # Save the filtered text to a new file in the Output folder
        filtered_text = ' '.join(tokens)
        with open(os.path.join(output_folder_path, filename), 'w', encoding='utf-8') as file:
            file.write(filtered_text)

# Save the results to a CSV file
results.to_csv('/content/drive/MyDrive/Results.csv', index=False)


<ipython-input-14-226929456054>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-14-226929456054>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-14-226929456054>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-14-226929456054>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-14-226929456054>:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-i

Note: Output folder contains text files after stopwords removal from the nltk library, as specified for the word count.

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from oauth2client.client import GoogleCredentials
gauth.credentials = GoogleCredentials.get_application_default()




---
**Performing readability analysis on the txt files and storing the metrics into a csv file.**


In [ ]:
import os
import csv
import string
import nltk
import glob
from textstat import flesch_reading_ease
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')

# Function to calculate the average sentence length
def avg_sentence_length(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    return len(words) / len(sentences)

# Function to calculate the percentage of complex words
def percent_complex_words(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if len(word) > 2 and word not in stopwords.words('english')]
    return len(complex_words) / len(words)

# Function to calculate the Fog Index
def fog_index(text):
    return 0.4 * (avg_sentence_length(text) + percent_complex_words(text))

# Function to calculate the average number of words per sentence
def avg_words_per_sentence(text):
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    return len(words) / len(sentences)

# Function to count the number of complex words
def complex_word_count(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if len(word) > 2 and word not in stopwords.words('english')]
    return len(complex_words)

# Function to count the total number of words
def word_count(text):
    words = word_tokenize(text)
    cleaned_words = [word.lower().translate(str.maketrans('', '', string.punctuation)) for word in words if word not in stopwords.words('english')]
    return len(cleaned_words)

# Function to count the number of syllables in each word
def syllable_count(word):
    vowels = 'aeiou'
    syllables = 0
    if word[0] in vowels:
        syllables += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i-1] not in vowels:
            syllables += 1
        # Handle exceptions
        if word.endswith('es') or word.endswith('ed'):
            syllables -= 1
        if word.endswith('le') and word[-2] not in vowels:
            syllables += 1
    # Handle single letter words
    if len(word) == 1:
        syllables = 1
    return syllables



# Function to calculate the average word length
def avg_word_length(text):
    words = word_tokenize(text)
    total_characters = sum(len(word) for word in words)
    return total_characters / len(words)




def personal_pronouns(text):
    count = 0
    personal_pronouns_ = ['I', 'we', 'my', 'our', 'ours', 'us','We', 'My', 'Our', 'Ours', 'Us']
    for word in text.split():
        if word in personal_pronouns_:
            count += 1
    return count

stop_words = set(stopwords.words('english'))
folder_path = os.path.abspath('/content/drive/MyDrive/URL_ID')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
with open('results4.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'Avg Sentence Length', 'Percentage of Complex Words', 'FOG Index', 'Avg Number of Words per Sentence', 'Complex Word Count', 'Word Count', 'Syllables per Word', 'Personal Pronouns', 'Avg Word Length'])

    files = glob.glob(os.path.join(folder_path, '*.txt'))
    for filename in files:
        print(filename)
        with open(filename, 'r') as f:
            text = f.read()
            sentences = nltk.sent_tokenize(text)
            total_words = len(nltk.word_tokenize(text))
            total_sentences = len(sentences)
            total_syllables = 0
            complex_word_count = 0
            personal_pronouns_ = personal_pronouns(text)
            for word in nltk.word_tokenize(text):
                if word not in stop_words and not word.isnumeric():
                    if len(word) > 2 and syllable_count(word) > 2:
                        complex_word_count += 1
                    total_syllables += syllable_count(word)
            avg_sen_len = total_words / total_sentences
            per_complex_words = complex_word_count / total_words
            fog_index = 0.4 * (avg_sen_len + per_complex_words)
            avg_words_per_sen = total_words / total_sentences
            word_count = len([word for word in nltk.word_tokenize(text) if word not in stop_words and not word.isnumeric() and word.isalpha()])
            syllables_per_word = total_syllables / word_count
            avg_word_len = sum(len(word) for word in nltk.word_tokenize(text) if word not in stop_words and not word.isnumeric() and word.isalpha()) / word_count
            writer.writerow([filename, round(avg_sen_len, 2), round(per_complex_words * 100, 2), round(fog_index, 2), round(avg_words_per_sen, 2), complex_word_count, word_count, round(syllables_per_word, 2), personal_pronouns_, round(avg_word_len, 2)])
            
            print(f"File {filename} completed.")


/content/drive/MyDrive/URL_ID/37.txt
File /content/drive/MyDrive/URL_ID/37.txt completed.
/content/drive/MyDrive/URL_ID/38.txt
File /content/drive/MyDrive/URL_ID/38.txt completed.
/content/drive/MyDrive/URL_ID/39.txt
File /content/drive/MyDrive/URL_ID/39.txt completed.
/content/drive/MyDrive/URL_ID/40.txt
File /content/drive/MyDrive/URL_ID/40.txt completed.
/content/drive/MyDrive/URL_ID/41.txt
File /content/drive/MyDrive/URL_ID/41.txt completed.
/content/drive/MyDrive/URL_ID/42.txt
File /content/drive/MyDrive/URL_ID/42.txt completed.
/content/drive/MyDrive/URL_ID/43.txt
File /content/drive/MyDrive/URL_ID/43.txt completed.
/content/drive/MyDrive/URL_ID/44.txt
File /content/drive/MyDrive/URL_ID/44.txt completed.
/content/drive/MyDrive/URL_ID/45.txt
File /content/drive/MyDrive/URL_ID/45.txt completed.
/content/drive/MyDrive/URL_ID/46.txt
File /content/drive/MyDrive/URL_ID/46.txt completed.
/content/drive/MyDrive/URL_ID/47.txt
File /content/drive/MyDrive/URL_ID/47.txt completed.
/content/d

In [ ]:
for filename in files:
  print(1)

In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Snippet to download the result csv of readability analysis.**

In [ ]:
from google.colab import files

# replace "results.csv" with the name of your file
files.download("results4.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Merging the csv files of sentiment analysis and preparing the final csv.**

In [ ]:
# Read input.xlsx and extract the columns we want
input_df = pd.read_excel('/content/drive/MyDrive/Input.xlsx')
input_df = input_df[['URL_ID', 'URL']]

# Read Results.csv and extract the columns we want
results_df = pd.read_csv('/content/part1result.csv')
results_df = results_df[['URL_ID', 'Positive Score', 'Negative Score', 'Polarity Score', 'Subjectivity Score']]

# Read results2.csv and extract the columns we want
results2_df = pd.read_csv('/content/part2result.csv')
results2_df = results2_df[['URL_ID','Avg Sentence Length', 'Percentage of Complex Words', 'FOG Index', 'Avg Number of Words per Sentence', 'Complex Word Count', 'Word Count', 'Syllables per Word', 'Personal Pronouns', 'Avg Word Length']]

# Merge the dataframes together on url_id
merged_df = input_df.merge(results_df, on='URL_ID').merge(results2_df, on='URL_ID')

# Write the final dataframe to a CSV file
merged_df.to_csv('finalresult.csv', index=False)

**Download the final csv**

In [ ]:
files.download("finalresult.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>